In [5]:
import os
import pandas as pd
import re
import numpy as np

# Determining the path where the files are located and creating a new folder
path = '/content/drive/MyDrive/BIMJSON/ProjectRevitIFC'
try:
  os.mkdir(path + '/quick_rough_estimate')
except:
  pass

In [14]:
# Properties for which we want to collect data on the amount of volume
propstr = ['Area', 'Volume', 'Width', 'Length', 'Perimeter', 'öööasdöööfake']

# The type of property by which the grouping will be carried out
search_parameter = 'Type Name'

# Main function for grouping data and saving a file
def crtable(filename):
    df = pd.read_csv(path + '/' + filename, low_memory=False)
    
    #  Fetching only numbers from string values of volumetric parameters
    propindf = []
    for el in propstr:
      if el in df.columns:
        propindf.append(el)
    def find_number(text):
      num = re.findall(r'[0-9]+', text)
      return ".".join(num)
    for el in propindf:
      df[el] = df[el].astype(str)
      df[el] = df[el].apply(lambda x: find_number(x))
      df[el] = pd.to_numeric(df[el], errors='coerce')
      df[el] = df[el].replace(np.nan, 0)
      df[el] = df[el].replace('None', 0)
      df[el] = df[el].fillna(0)
    try:
        df[el] = df[el].astype(float)
    except:
        pass

    # Summation of all data that are grouped by search_parameter located in the propindf columns
    df1=pd.pivot_table(df, index=[search_parameter],values=propindf,aggfunc=np.sum)
    df1 = df1.add_prefix('Sum of ')

    # Determination of the number of elements in groups
    df2= df.groupby([search_parameter])[propindf[0]].agg(['count'])
    dfallpar = pd.DataFrame()  
    df['Unnamed: 0'] = df['Unnamed: 0'].astype(str)
    comma = lambda x: ', '.join(x.unique())
    df3 = df.groupby([search_parameter]) .agg({'Unnamed: 0': comma})

    """
    for el in propindf:
      df_groups_vol= df.groupby([search_parameter])[el].agg(['sum'])
      dfallpar = dfallpar.append(df_groups_vol)
    """
    # Collecting data into one dataframe
    dfallpar = pd.concat([df2, df1, df3], axis=1)
    dfallpar.rename(columns=({ 'Unnamed: 0': 'Id´s', 'count': 'Amount'}), inplace=True,)

    # Saving data to file
    dfallpar.to_excel(path + '/quick_rough_estimate/' + 'QRE_' + filename+'.xlsx',  sheet_name='Rough Estimate')

for filename in os.listdir(path):
    if filename.endswith("csv"): 
        crtable(filename)    
        break
